In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import networkx as nx
import pathlib
import pickle
from numba import jit

In [ ]:
import importlib
import evotsc
import evotsc_lib
import evotsc_plot
importlib.reload(evotsc)
importlib.reload(evotsc_lib)
importlib.reload(evotsc_plot)

In [ ]:
exp_path = pathlib.Path('/Users/theotime/Desktop/evotsc/pci/main/')
gen=1_000_000
gene_types = ['AB', 'A', 'B'] # Name of each gene type
gene_type_color = ['tab:blue', 'tab:red', 'tab:green'] #AB, A, B
envs = ['A', 'B']
dpi=300

In [ ]:
rep_dirs = sorted([d for d in exp_path.iterdir() if (d.is_dir() and d.name.startswith("rep"))])
params = evotsc_lib.read_params(rep_dirs[0])
params['m'] = 2.5 # Temporary fix because the parameter wasn't saved

In [ ]:
rng = np.random.default_rng(seed=123456)

## Arrange Individual and run_system() to accommodate a knocked-out gene

In [ ]:
@jit(nopython=True)
def run_system_numba_ko(nb_genes: int,
                        init_expr: np.ndarray,
                        inter_matrix: np.ndarray,
                        sigma_basal: float,
                        sigma_opt: float,
                        epsilon: float,
                        m: float,
                        sigma_env: float,
                        id_ko: int) -> np.ndarray:

    step_size = 0.5
    stop_dist = 1e-7
    max_eval_steps = 200

    temporal_expr = np.zeros((max_eval_steps+1, nb_genes))

    # Initial values at t = 0
    temporal_expr[0, :] = init_expr
    temporal_expr[0, id_ko] = 0.0


    # Iterate the system
    it = 1
    cont = True
    while cont:
        prev_expr = temporal_expr[it-1, :]
        sigma_local = inter_matrix @ prev_expr
        sigma_total = sigma_basal + sigma_local + sigma_env

        promoter_activity = 1.0 / (1.0 + np.exp((sigma_total - sigma_opt)/epsilon))

        # We subtract 1 to rescale between exp(-m) and 1
        iter_expr = np.exp(m * (promoter_activity - 1.0))

        nouv_expr = step_size * iter_expr + (1 - step_size) * prev_expr

        # Knockout
        nouv_expr[id_ko] = 0

        temporal_expr[it, :] = nouv_expr

        # Check if we're done
        dist = np.abs(nouv_expr - prev_expr).sum() / nb_genes

        prev_expr = nouv_expr

        if dist < stop_dist:
            cont = False

        if it == max_eval_steps:
            cont = False
        it += 1

    temporal_expr = temporal_expr[:it, :]

    return temporal_expr

In [ ]:
class KoIndividual(evotsc.Individual):
    def __init__(self,
                 orig_indiv: evotsc.Individual,
                 id_ko: int) -> None:
        
        super().__init__([gene.clone() for gene in orig_indiv.genes],
                         orig_indiv.interaction_dist,
                         orig_indiv.interaction_coef,
                         orig_indiv.sigma_basal,
                         orig_indiv.sigma_opt,
                         orig_indiv.epsilon,
                         orig_indiv.m,
                         orig_indiv.selection_coef,
                         orig_indiv.rng)
        
        self.id_ko = id_ko
                    

    def run_system(self, sigma_env):

        init_expr = np.array([gene.basal_expression for gene in self.genes])

        self.inter_matrix = self.compute_inter_matrix()

        return run_system_numba_ko(nb_genes=self.nb_genes,
                                   init_expr=init_expr,
                                   inter_matrix=self.inter_matrix,
                                   sigma_basal=self.sigma_basal,
                                   sigma_opt=self.sigma_opt,
                                   epsilon=self.epsilon,
                                   m=self.m,
                                   sigma_env=sigma_env,
                                   id_ko=self.id_ko)

## Plot genome and TSC with one knocked-out gene

In [ ]:
def plot_all_kos(rep, gen):
    
    rep_dir = exp_path.joinpath(f'rep{rep}')
    
    ko_path = exp_path.joinpath(f'kos_rep{rep:02}')
    ko_path.mkdir(exist_ok=True)
    
    init_indiv = evotsc_lib.get_best_indiv(rep_dir, gen=gen)
    n = 100  # the number of data points
    _, genome_length = init_indiv.compute_gene_positions(include_coding=True)
    data_positions = np.linspace(0, genome_length, n, dtype=int)

    # Note: we do the difference between the supercoiling values of two genomes of the same individual,
    # so sigma_basal and sigma_env cancel out
    init_data = {'A': init_indiv.compute_final_sc_at(params['sigma_A'], data_positions),
                 'B': init_indiv.compute_final_sc_at(params['sigma_B'], data_positions)}
    
    init_expr = {'A': init_indiv.run_system(params['sigma_A'])[-1, :],
                 'B': init_indiv.run_system(params['sigma_B'])[-1, :]}
    
    
    half_expr = (1 + np.exp(-init_indiv.m)) / 2
    init_activ = {env: init_expr[env] > half_expr for env in envs}
    
    #for i_ko in range(36, 37): 
    for i_ko in range(init_indiv.nb_genes):
        
        ko_indiv = KoIndividual(orig_indiv=init_indiv, id_ko=i_ko)
                                            
        for env in envs:
                                            
            ko_data = ko_indiv.compute_final_sc_at(params[f'sigma_{env}'], data_positions)
            
            final_expr = ko_indiv.run_system(params[f'sigma_{env}'])[-1, :]
            ko_activ = final_expr > half_expr
            
            switched_genes = init_activ[env] != ko_activ
            
            show_bar = env == 'A'
            
            plot_name = ko_path.joinpath(f'ko_genome_and_tsc_env_{env}_gene_{i_ko:02}.pdf')
            
            evotsc_plot.plot_genome_and_tsc(ko_indiv, sigma=params[f'sigma_{env}'], coloring_type='on-off',
                                            hatched_genes=switched_genes,
                                            print_ids=True, ring_data=np.abs(init_data[env]-ko_data), 
                                            ring_color_type='delta', 
                                            show_bar=show_bar, bar_text='$|\Delta\sigma_{TSC}|$',
                                            id_interval=5, id_ko=i_ko,
                                            plot_name=plot_name)

In [ ]:
plot_all_kos(rep=21, gen=gen)

## Compute the effective graph: genes switched on or off by a KO

In [ ]:
def get_effective_graph(best_indiv, sigma):
    
    _, genome_length = best_indiv.compute_gene_positions(include_coding=True)
    
    init_expr = best_indiv.run_system(sigma)[-1, :]
    half_expr = (1 + np.exp(-best_indiv.m)) / 2
    init_activ = init_expr > half_expr
    

    ## Build the graph
    inter_graph = nx.DiGraph()

    # Nodes
    for i_gene, gene in enumerate(best_indiv.genes):
        inter_graph.add_node(i_gene, gene=gene)

    ## Compute the interactions: if KOing gene A switches gene B on or off, add A -> B arrow to the graph

    for i_ko in range(best_indiv.nb_genes):
        
        ko_indiv = KoIndividual(orig_indiv=best_indiv, id_ko=i_ko)
                                                                    
        final_expr = ko_indiv.run_system(sigma)[-1, :]
        ko_activ = final_expr > half_expr
        
        for i_other in range(best_indiv.nb_genes):
            if i_other == i_ko:
                continue
            
            if init_activ[i_other] != ko_activ[i_other]:
                if init_activ[i_other]: # KOing i_ko inhibits i_other: -> i_ko activates i_other
                    inter_graph.add_edge(i_ko, i_other, kind='activ')
                else:
                    inter_graph.add_edge(i_ko, i_other, kind='inhib')

    return best_indiv, inter_graph

In [ ]:
def get_all_effective_graphs(exp_path, gen):
    rep_dirs = sorted([d for d in exp_path.iterdir() if (d.is_dir() and d.name.startswith("rep"))])
    
    graphs = {rep: {} for rep in range(len(rep_dirs))}
    
    for rep, rep_dir in enumerate(rep_dirs):
        best_indiv = evotsc_lib.get_best_indiv(rep_path=rep_dir, gen=gen)
        for env in ['A', 'B']:
            graphs[rep][env] = get_effective_graph(best_indiv, params[f'sigma_{env}'])
    
    return graphs

In [ ]:
effective_graphs = get_all_effective_graphs(exp_path, gen)

## Plot genomes again (with interaction graphs inside)

In [ ]:
def plot_best_genome_and_tsc(exp_path, gen, effective_graphs):
    rep_dirs = sorted([d for d in exp_path.iterdir() if (d.is_dir() and d.name.startswith("rep"))])
    
    genome_path = exp_path.joinpath(f'genomes')
    genome_path.mkdir(exist_ok=True)
    
    for rep in effective_graphs.keys():
        
        best_indiv, graph_A = effective_graphs[rep]['A']
        best_indiv, graph_B = effective_graphs[rep]['B']

        evotsc_plot.plot_genome_and_tsc(best_indiv, params['sigma_A'], show_bar=True,
                                        coloring_type='type',
                                        id_interval=best_indiv.nb_genes / 12, print_ids=True,
                                        inter_graph=nx.compose(graph_A, graph_B),
                                    plot_name=genome_path.joinpath(f'genome_and_tsc_rep{rep:02}_combined.pdf'))


In [ ]:
plot_best_genome_and_tsc(exp_path, gen, effective_graphs)

In [ ]:
def make_random_effective_graphs(nb_indiv):
    
    graphs = {rep: {} for rep in range(nb_indiv)}
    
    mutation = evotsc.Mutation(inversion_poisson_lam=params['inversion_poisson_lam'])
    
    for rep in range(nb_indiv):
        indiv = evotsc_lib.make_random_indiv(intergene=int(params['intergene']),
                                             gene_length=int(params['gene_length']),
                                             nb_genes=int(params['nb_genes']),
                                             default_basal_expression=params['default_basal_expression'],
                                             interaction_dist=params['interaction_dist'],
                                             interaction_coef=params['interaction_coef'],
                                             sigma_basal=params['sigma_basal'],
                                             sigma_opt=params['sigma_opt'],
                                             epsilon=params['epsilon'],
                                             m=params['m'],
                                             selection_coef=params['selection_coef'],
                                             mutation=mutation,
                                             rng=rng,
                                             nb_mutations=100)
        
        indiv.inter_matrix = indiv.compute_inter_matrix()

        for env in ['A', 'B']:
            graphs[rep][env] = get_effective_graph(indiv, params[f'sigma_{env}'])
    
    return graphs

In [ ]:
rand_effective_graphs = make_random_effective_graphs(nb_indiv=30)

## Plot the effective graphs

In [ ]:
def plot_effective_graph(indiv, inter_graph, method, plot_name=None):
    
    ## Draw the figure
    plt.figure(figsize=(12, 12), dpi=dpi)
    plt.box(False)

    # Choose the layout
    if method == 'circular':
        layout = nx.circular_layout(inter_graph)
        for node in layout:
            x, y = layout[node]
            layout[node] = y, x # symmetry along the y=x axis to match other graphs

    elif method == 'graphviz':
        layout = nx.nx_agraph.graphviz_layout(inter_graph)
        for node in layout:
            x, y = layout[node]
            layout[node] = -x, -y # 180 degree rotation


    # Draw the nodes
    node_size = 400
    nx.draw_networkx_nodes(inter_graph, layout, node_size=node_size,
                           node_color=[gene_type_color[gene.gene_type] for gene in indiv.genes])
    nx.draw_networkx_labels(inter_graph, layout)

    # Draw the edges
    activ_edges = [e for e in inter_graph.edges if inter_graph[e[0]][e[1]]['kind'] == 'activ'] 
    inhib_edges = [e for e in inter_graph.edges if inter_graph[e[0]][e[1]]['kind'] == 'inhib']

    inhib_arrow = mpl.patches.ArrowStyle.CurveFilledB(head_length=0, head_width=0.5)

    nx.draw_networkx_edges(inter_graph, layout, edgelist=activ_edges, node_size=node_size,
                           edge_color='tab:green', connectionstyle='arc3,rad=0.1')
    nx.draw_networkx_edges(inter_graph, layout, edgelist=inhib_edges, node_size=node_size+75,
                           arrowstyle=inhib_arrow, edge_color='tab:red',
                           connectionstyle='arc3,rad=0.1')

    if plot_name:
        plt.savefig(plot_name, bbox_inches='tight', dpi=dpi)
    plt.show()
    plt.close()

In [ ]:
def plot_all_effective_graphs(effective_graphs, method, is_random=False):
        
    graph_path = exp_path.joinpath(f'effective_graphs')
    graph_path.mkdir(exist_ok=True)
    
    for rep in list(effective_graphs.keys()):
        
        for env in ['A', 'B']:

            indiv, inter_graph = effective_graphs[rep][env]
            
            plot_name = f'effective_graph_rep{rep:02}_env_{env}_{method}.pdf'
            if is_random:
                plot_name = f'rand_{plot_name}'
    
            full_name = graph_path.joinpath(plot_name)

            plot_effective_graph(indiv, inter_graph, method, full_name)

In [ ]:
#plot_all_effective_graphs(effective_graphs, method='circular')
#plot_all_effective_graphs(effective_graphs, method='graphviz')

In [ ]:
#plot_all_effective_graphs(rand_effective_graphs, method='circular', is_random=True)
#plot_all_effective_graphs(rand_effective_graphs, method='graphviz', is_random=True)

In [ ]:
def plot_combined_effective_graphs(effective_graphs, method, is_random=False):
    graph_path = exp_path.joinpath(f'effective_graphs')
    graph_path.mkdir(exist_ok=True)
    
    for rep in list(effective_graphs.keys()):
        indiv, graph_A = effective_graphs[rep]['A']
        indiv, graph_B = effective_graphs[rep]['B']
        
        combined_graph = nx.compose(graph_A, graph_B)
        
        plot_name = f'combined_effective_graph_rep{rep:02}_{method}.pdf'
        if is_random:
            plot_name = f'rand_{plot_name}'

        full_name = graph_path.joinpath(plot_name)
        
        plot_effective_graph(indiv, combined_graph, method, full_name)

In [ ]:
plot_combined_effective_graphs(effective_graphs, 'graphviz', is_random=False)

In [ ]:
#plot_combined_effective_graphs(rand_effective_graphs, 'graphviz', is_random=True)

## Connected components

In [ ]:
def count_largest_wcc(effective_graphs):
    
    largest_wcc = np.zeros(len(effective_graphs))
    
    for rep in list(effective_graphs.keys()):
        indiv, graph_A = effective_graphs[rep]['A']
        indiv, graph_B = effective_graphs[rep]['B']

        combined_graph = nx.compose(graph_A, graph_B)
        
        largest_wcc[rep] = len(max(nx.weakly_connected_components(combined_graph), key=len))
        
    return largest_wcc

In [ ]:
def plot_largest_wccs(effective_graphs, rand_effective_graphs):

    evolved_wccs = count_largest_wcc(effective_graphs)

    rand_wccs = count_largest_wcc(rand_effective_graphs)

    plt.figure(figsize=(7, 4), dpi=300)
    plt.violinplot([evolved_wccs, rand_wccs])
    plt.xticks(ticks=[1, 2], labels=['Evolved', 'Random'])
    plt.ylim(-3, 63)
    plt.ylabel('Number of genes in the largest WCC')
    plt.grid(linestyle=':', axis='y')
    plt.show()

In [ ]:
plot_largest_wccs(effective_graphs, rand_effective_graphs)

In [ ]:
def gather_all_wccs(effective_graphs):
    wccs = []
    
    for rep in list(effective_graphs.keys()):
        indiv, graph_A = effective_graphs[rep]['A']
        indiv, graph_B = effective_graphs[rep]['B']

        combined_graph = nx.compose(graph_A, graph_B)
        
        for wcc in nx.weakly_connected_components(combined_graph):
            wccs.append(len(wcc))
                
    return np.array(wccs)

In [ ]:
def plot_all_wccs(effective_graphs, rand_effective_graphs):

    evolved_wccs = gather_all_wccs(effective_graphs)

    rand_wccs = gather_all_wccs(rand_effective_graphs)

    plt.figure(figsize=(8, 5), dpi=dpi)
    plt.boxplot([evolved_wccs, rand_wccs], medianprops={'color':'black'})
    plt.xticks(ticks=[1, 2], labels=['Evolved', 'Random'])
    plt.ylim(-3, 63)
    plt.ylabel('Number of genes in the WCC')
    plt.grid(linestyle=':', axis='y')
    plt.savefig(exp_path.joinpath('effective_graph_wcc_distr.pdf'), bbox_inches='tight', dpi=dpi)
    plt.show()

In [ ]:
plot_all_wccs(effective_graphs, rand_effective_graphs)

### Show the exact count for every WCC size

In [ ]:
np.unique(gather_all_wccs(effective_graphs), return_counts=True)

In [ ]:
np.unique(gather_all_wccs(rand_effective_graphs), return_counts=True)

## Stats on the effective graphs: out degree (number of switches) and farthest switch

In [ ]:
def dist_in_genes(nb_genes, i, j):
    pos_1_minus_2 = i - j
    pos_2_minus_1 = - pos_1_minus_2

    # We want to know whether gene 1 comes before or after gene 2
    # Before: -------1--2-------- or -2---------------1-
    # After:  -------2--1-------- or -1---------------2-

    if pos_1_minus_2 < 0: # -------1--2-------- ou -1---------------2-
        if pos_2_minus_1 < nb_genes + pos_1_minus_2: # -------1--2--------
            distance = pos_2_minus_1
        else: # -1---------------2-
            distance = nb_genes + pos_1_minus_2

    else: # -------2--1-------- ou -2---------------1-
        if pos_1_minus_2 < nb_genes + pos_2_minus_1: # -------2--1--------
            distance = pos_1_minus_2
        else:
            distance = nb_genes + pos_2_minus_1

    return distance

In [ ]:
def get_effective_graph_stats(effective_graphs):
    res_dict = {'Replicate': [],
                'gene_type': [],
                'env': [],
                'out_deg': [],
                'avg_dist': [],
                'max_dist': []}

    for rep in effective_graphs.keys():
        for env in ['A', 'B']:
            indiv, graph = effective_graphs[rep][env]
            for i_gene, gene in enumerate(indiv.genes):
                res_dict['Replicate'].append(rep)
                res_dict['gene_type'].append(gene_types[gene.gene_type])
                res_dict['env'].append(env)
                res_dict['out_deg'].append(graph.out_degree[i_gene])
                
                if graph.out_degree[i_gene] > 0:
                    dists = np.zeros(len(graph[i_gene].keys()), dtype=int)                
                    for i_neighbor, neighbor in enumerate(graph[i_gene].keys()):
                        dists[i_neighbor] = dist_in_genes(indiv.nb_genes, i_gene, neighbor)

                    res_dict['avg_dist'].append(np.mean(dists))
                    res_dict['max_dist'].append(np.max(dists))
                
                else:
                    res_dict['avg_dist'].append(0)
                    res_dict['max_dist'].append(0)
                
    return pd.DataFrame.from_dict(res_dict)

In [ ]:
graph_stats = get_effective_graph_stats(effective_graphs)

In [ ]:
rand_graph_stats = get_effective_graph_stats(rand_effective_graphs)

In [ ]:
def get_combined_graph_stats(effective_graphs):
    res_dict = {'Replicate': [],
                'gene_type': [],
                'out_deg': [],
                'avg_dist': [],
                'max_dist': []}

    for rep in effective_graphs.keys():

        indiv, graph_A = effective_graphs[rep]['A']
        indiv, graph_B = effective_graphs[rep]['B']
        combined_graph = nx.compose(graph_A, graph_B)
        
        for i_gene, gene in enumerate(indiv.genes):
            res_dict['Replicate'].append(rep)
            res_dict['gene_type'].append(gene_types[gene.gene_type])
            res_dict['out_deg'].append(combined_graph.out_degree[i_gene])

            if combined_graph.out_degree[i_gene] > 0:
                dists = np.zeros(len(combined_graph[i_gene].keys()), dtype=int)                
                for i_neighbor, neighbor in enumerate(combined_graph[i_gene].keys()):
                    dists[i_neighbor] = dist_in_genes(indiv.nb_genes, i_gene, neighbor)

                res_dict['avg_dist'].append(np.mean(dists))
                res_dict['max_dist'].append(np.max(dists))

            else:
                res_dict['avg_dist'].append(0)
                res_dict['max_dist'].append(0)
                
    return pd.DataFrame.from_dict(res_dict)

In [ ]:
combined_stats = get_combined_graph_stats(effective_graphs)

In [ ]:
combined_rand_stats = get_combined_graph_stats(rand_effective_graphs)

## Plot the average out degree of nodes in the network

In [ ]:
def plot_out_degree_by_env(graph_stats, rand_graph_stats):
    
    fig, ax = plt.subplots(figsize=(8, 5), dpi=300)

    mean_stats = graph_stats.groupby(['env', 'gene_type']).mean()['out_deg']
    rep_stats = graph_stats.groupby(['gene_type', 'env', 'Replicate']).mean()['out_deg']
    rand_mean_stats = rand_graph_stats.groupby(['env', 'gene_type']).mean()['out_deg']
    rand_rep_stats = rand_graph_stats.groupby(['gene_type', 'env', 'Replicate']).mean()['out_deg']
    
    width = 0.2
    x_pos = np.array([0, 4*width, 9*width, 13*width])
    delta = np.array([-width, 0, width])

    rects = {}

    for i_gene_type, gene_type in enumerate(gene_types):
        rects[gene_type] = plt.bar(x_pos + delta[i_gene_type],
                                   [mean_stats.loc['A'][gene_type], rand_mean_stats.loc['A'][gene_type],
                                    mean_stats.loc['B'][gene_type], rand_mean_stats.loc['B'][gene_type]],
                                    width=width, color=gene_type_color[i_gene_type])

        plt.boxplot([rep_stats.loc[(gene_type, 'A')], rand_rep_stats.loc[(gene_type, 'A')],
                     rep_stats.loc[(gene_type, 'B')], rand_rep_stats.loc[(gene_type, 'B')]],
                    positions=x_pos + delta[i_gene_type], 
                manage_ticks=False, widths=0.1, medianprops={'color':'black'})

    plt.xticks(ticks=x_pos, labels=['Evolved env. A', 'Random env. A', 'Evolved env. B', 'Random env. B'])
    ax.yaxis.set_major_locator(mpl.ticker.MultipleLocator(1))

    plt.ylabel(f'Out degree in the effective graph')
    plt.ylim(0, 11)
    plt.grid(axis='y', linestyle=':')
    
    patches = ([mpl.patches.Patch(facecolor=color, edgecolor='black', label=label)
                for color, label in zip(gene_type_color, gene_types)])
    plt.legend(handles=patches, title='Gene type')#, title_fontsize=15, fontsize=15)

    
    plt.savefig(exp_path.joinpath('effective_graph_out_degree_by_env.pdf'), bbox_inches='tight', dpi=dpi)

In [ ]:
plot_out_degree_by_env(graph_stats, rand_graph_stats)

### Do the same for the combined graphs

In [ ]:
def plot_out_degree_by_env_combined(graph_stats, rand_graph_stats):
    
    fig, ax = plt.subplots(figsize=(8, 5), dpi=300)

    mean_stats = graph_stats.groupby(['gene_type']).mean()['out_deg']
    rep_stats = graph_stats.groupby(['gene_type', 'Replicate']).mean()['out_deg']
    rand_mean_stats = rand_graph_stats.groupby(['gene_type']).mean()['out_deg']
    rand_rep_stats = rand_graph_stats.groupby(['gene_type', 'Replicate']).mean()['out_deg']
    
    width = 0.2
    x_pos = np.array([1, 2])
    delta = np.array([-width, 0, width])

    rects = {}

    for i_gene_type, gene_type in enumerate(gene_types):
        rects[gene_type] = plt.bar(x_pos + delta[i_gene_type],
                                   [mean_stats.loc[gene_type], rand_mean_stats.loc[gene_type]],
                                    width=width, color=gene_type_color[i_gene_type])

        plt.boxplot([rep_stats.loc[gene_type], rand_rep_stats.loc[gene_type]],
                    positions=x_pos + delta[i_gene_type], 
                    #manage_ticks=False, 
                    widths=0.1, medianprops={'color':'black'})

    plt.xticks(ticks=x_pos, labels=['Evolved', 'Random'])
    ax.yaxis.set_major_locator(mpl.ticker.MultipleLocator(1))

    plt.ylabel(f'Out degree in the effective interaction graph')
    plt.ylim(0, 11)
    plt.grid(axis='y', linestyle=':')
    
    patches = ([mpl.patches.Patch(facecolor=color, edgecolor='black', label=label)
                for color, label in zip(gene_type_color, gene_types)])
    plt.legend(handles=patches, title='Gene type')#, title_fontsize=15, fontsize=15)

    
    plt.savefig(exp_path.joinpath('effective_graph_combined_out_degree.pdf'), bbox_inches='tight', dpi=dpi)

In [ ]:
plot_out_degree_by_env_combined(combined_stats, combined_rand_stats)

## Plot the mean distance to switched genes

In [ ]:
def plot_avg_dist_by_env(graph_stats, rand_graph_stats):
    
    fig, ax = plt.subplots(figsize=(8, 5), dpi=300)
    
    graph_stats = graph_stats[graph_stats['out_deg'] > 0]
    rand_graph_stats = rand_graph_stats[rand_graph_stats['out_deg'] > 0]

    mean_stats = graph_stats.groupby(['env', 'gene_type']).mean()['avg_dist']
    rep_stats = graph_stats.groupby(['gene_type', 'env', 'Replicate']).mean()['avg_dist']
    rand_mean_stats = rand_graph_stats.groupby(['env', 'gene_type']).mean()['avg_dist']
    rand_rep_stats = rand_graph_stats.groupby(['gene_type', 'env', 'Replicate']).mean()['avg_dist']
    
    width = 0.2
    x_pos = np.array([0, 4*width, 9*width, 13*width])
    delta = np.array([-width, 0, width])

    rects = {}

    for i_gene_type, gene_type in enumerate(gene_types):
        rects[gene_type] = plt.bar(x_pos + delta[i_gene_type],
                                   [mean_stats.loc['A'][gene_type], rand_mean_stats.loc['A'][gene_type],
                                    mean_stats.loc['B'][gene_type], rand_mean_stats.loc['B'][gene_type]],
                                    width=width, color=gene_type_color[i_gene_type])

        plt.boxplot([rep_stats.loc[(gene_type, 'A')], rand_rep_stats.loc[(gene_type, 'A')],
                     rep_stats.loc[(gene_type, 'B')], rand_rep_stats.loc[(gene_type, 'B')]],
                    positions=x_pos + delta[i_gene_type], 
                manage_ticks=False, widths=0.1, medianprops={'color':'black'})

    plt.xticks(ticks=x_pos, labels=['Evolved env. A', 'Random env. A', 'Evolved env. B', 'Random env. B'])
    ax.yaxis.set_major_locator(mpl.ticker.MultipleLocator(1))

    plt.ylabel(f'Mean distance to neighbors in the effective graph')
    plt.ylim(0, 11)
    plt.grid(axis='y', linestyle=':')
    
    patches = ([mpl.patches.Patch(facecolor=color, edgecolor='black', label=label)
                for color, label in zip(gene_type_color, gene_types)])
    plt.legend(handles=patches, title='Gene type')#, title_fontsize=15, fontsize=15)

    plt.savefig(exp_path.joinpath('effective_graph_avg_dist_by_env.pdf'), bbox_inches='tight', dpi=dpi)

In [ ]:
plot_avg_dist_by_env(graph_stats, rand_graph_stats)

## Plot the max distance to a switched gene

In [ ]:
def plot_max_dist_by_env(graph_stats, rand_graph_stats):
    
    fig, ax = plt.subplots(figsize=(8, 5), dpi=300)

    # For each gene type, take the furthest switched gene by a gene of that type
    rep_stats = graph_stats.groupby(['gene_type', 'env', 'Replicate']).max()['max_dist']
    mean_stats = rep_stats.groupby(['env', 'gene_type']).mean() # Average the max of each replica
    rand_rep_stats = rand_graph_stats.groupby(['gene_type', 'env', 'Replicate']).max()['max_dist']
    rand_mean_stats = rand_rep_stats.groupby(['env', 'gene_type']).mean() # Average the max of each replica

    width = 0.2
    x_pos = np.array([0, 4*width, 9*width, 13*width])
    delta = np.array([-width, 0, width])

    rects = {}

    for i_gene_type, gene_type in enumerate(gene_types):
        rects[gene_type] = plt.bar(x_pos + delta[i_gene_type],
                                   [mean_stats.loc['A'][gene_type], rand_mean_stats.loc['A'][gene_type],
                                    mean_stats.loc['B'][gene_type], rand_mean_stats.loc['B'][gene_type]],
                                    width=width, color=gene_type_color[i_gene_type])

        plt.boxplot([rep_stats.loc[(gene_type, 'A')], rand_rep_stats.loc[(gene_type, 'A')],
                     rep_stats.loc[(gene_type, 'B')], rand_rep_stats.loc[(gene_type, 'B')]],
                    positions=x_pos + delta[i_gene_type], 
                manage_ticks=False, widths=0.1, medianprops={'color':'black'})

    plt.xticks(ticks=x_pos, labels=['Evolved env. A', 'Random env. A', 'Evolved env. B', 'Random env. B'])
    #ax.yaxis.set_major_locator(mpl.ticker.MultipleLocator(1))

    plt.ylabel(f'Furthest switch in the effective graph')
    plt.grid(axis='y', linestyle=':')
    
    patches = ([mpl.patches.Patch(facecolor=color, edgecolor='black', label=label)
                for color, label in zip(gene_type_color, gene_types)])
    plt.legend(handles=patches, title='Gene type')#, title_fontsize=15, fontsize=15)

    
    plt.savefig(exp_path.joinpath('effective_graph_max_dist_by_env.pdf'), bbox_inches='tight', dpi=dpi)

In [ ]:
plot_max_dist_by_env(graph_stats, rand_graph_stats)